In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
sns.set(style="ticks")

In [40]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [43]:
len(train_data)

25000

In [4]:
train_data.shape

(25000,)

In [6]:
max([max(review) for review in train_data])

9999

In [8]:
# word_index is a dictionary mapping words to an integer index
word_index = imdb.get_word_index()
# We reverse it, mapping integer indices to words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [19]:
def give_review(review_index):
    review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[review_index]])
    return review

In [20]:
give_review(24999)

"? as a big fan of the original film it's hard to watch this show the ? set ? and ? ? sets rob any style from this remake the mood is never there instead it has the look and feel of so many television movies of the seventies crenna is not a bad choice as walter ? but his snappy wardrobe and ? apartment don't fit the mood of the original or make him an interesting character he does his best to make it work but samantha ? is a really bad choice the english accent and california looks can't hold a candle to barbara ? ? voice and sex appeal lee j ? tries ? to fashion barton ? but even his performance is just gruff without style br br it feels like the tv movie it was and again reminds me of what a remarkable film the original still is"

In [22]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in tqdm(enumerate(sequences)):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

# Our vectorized training data
x_train = vectorize_sequences(train_data)
# Our vectorized test data
x_test = vectorize_sequences(test_data)

C:\Users\48570\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [25]:
# vectorized labels
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [127]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# Create a tokenizer, configured to only take
# into account the top-1000 most common words
tokenizer = Tokenizer(num_words=1000)
# This builds the word index
tokenizer.fit_on_texts(samples)

# This turns strings into lists of integer indices.
sequences = tokenizer.texts_to_sequences(samples)
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

word_index = tokenizer.word_index
print('Found {} unique tokens.'.format(len(word_index)))

Found 9 unique tokens.


In [132]:
from keras.preprocessing.text import Tokenizer

reviews = [give_review(i) for i in range(len(train_data))]
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(reviews)

# This is how you can recover the word index that was computed
word_index = tokenizer.word_index
print('Found {} unique tokens.'.format(len(word_index)))

Found 9996 unique tokens.


In [67]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [70]:
from keras import losses
from keras import metrics

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

In [71]:
val_set_x = x_train[:10000]
the_rest_train_set_x = x_train[10000:]

val_set_y = y_train[:10000]
the_rest_train_set_y = y_train[10000:]

In [53]:
history = model.fit(the_rest_train_set_x,
                    the_rest_train_set_y,
                    epochs=20,
                    batch_size=512,
                    validation_data=(val_set_x, val_set_y))

Train on 15000 samples, validate on 10000 samples
Epoch 1/20
15000/15000 [==============================] - 3s 216us/step - loss: 0.5106 - binary_accuracy: 0.7811 - val_loss: 0.3901 - val_binary_accuracy: 0.8600
Epoch 2/20
15000/15000 [==============================] - 2s 139us/step - loss: 0.3024 - binary_accuracy: 0.9023 - val_loss: 0.3130 - val_binary_accuracy: 0.8818
Epoch 3/20
15000/15000 [==============================] - 2s 141us/step - loss: 0.2178 - binary_accuracy: 0.9323 - val_loss: 0.2854 - val_binary_accuracy: 0.8870
Epoch 4/20
15000/15000 [==============================] - 2s 136us/step - loss: 0.1713 - binary_accuracy: 0.9447 - val_loss: 0.2819 - val_binary_accuracy: 0.8862
Epoch 5/20
15000/15000 [==============================] - 2s 136us/step - loss: 0.1405 - binary_accuracy: 0.9539 - val_loss: 0.2914 - val_binary_accuracy: 0.8863
Epoch 6/20
15000/15000 [==============================] - 2s 137us/step - loss: 0.1149 - binary_accuracy: 0.9643 - val_loss: 0.3001 - val_bi

In [54]:
history_dict = history.history
history_dict.keys()

dict_keys(['val_loss', 'val_binary_accuracy', 'loss', 'binary_accuracy'])

In [114]:
results

[0.32499700770378115, 0.8728]

## In 88% cases our model can predict wheter the review is good or bad